In [1]:
import pandas as pd
import numpy as np

In [2]:
tmcs = pd.read_excel('data/predict_5_400.xlsx', sheet_name='TMCs', usecols=['compounds','CurrentDensity','overpotential']).loc[:16165, :]
print(tmcs.shape)
cols_tm = ['element','AN','AW','CR','AR','VR','AV','FIE','SIE','EA','EN','Nd','Nv','D','EC','TC']
tm = pd.read_excel('data/1.xlsx', sheet_name='TM_descriptors', usecols=cols_tm).fillna(0,inplace = False).replace('/', 0, inplace=False)
cols_c = ['element','AN','AW','CR','AR','VR','AV','FIE','SIE','EA','EN','Np','Nv','D']
c =  pd.read_excel('data/1.xlsx', sheet_name='C_descriptors', usecols=cols_c).fillna(0,inplace = False)

(16164, 3)


In [3]:
import re
import json

def parse_compound(compound):
    matches = re.findall(r'([A-Z][a-z]*)(\d*\.*\d*)', compound)
    result = {}
    for match in matches:
        element, count = match
        count = float(count) if count else 1.0  # 如果没有数字，默认为1.0
        result[element] = count
    
    return result

def get_res(compound_name):
    parsed_result = parse_compound(compound_name)
    return parsed_result

In [4]:
test = get_res('Mn0.2Cr0.8N')

In [5]:
tm.columns.tolist()

['element',
 'AN',
 'AW',
 'CR',
 'AR',
 'VR',
 'AV',
 'FIE',
 'SIE',
 'EA',
 'EN',
 'Nd',
 'Nv',
 'D',
 'EC',
 'TC']

In [6]:
c.columns.tolist()

['element',
 'AN',
 'AW',
 'CR',
 'AR',
 'VR',
 'AV',
 'FIE',
 'SIE',
 'EA',
 'EN',
 'Np',
 'Nv',
 'D']

In [7]:
columns = [f'AB_{i}' for i in tm.columns.tolist()[1:]] + [f'C_{i}' for i in c.columns.tolist()[1:]] + ['CurrentDensity','overpotential']
np.array(columns)

array(['AB_AN', 'AB_AW', 'AB_CR', 'AB_AR', 'AB_VR', 'AB_AV', 'AB_FIE',
       'AB_SIE', 'AB_EA', 'AB_EN', 'AB_Nd', 'AB_Nv', 'AB_D', 'AB_EC',
       'AB_TC', 'C_AN', 'C_AW', 'C_CR', 'C_AR', 'C_VR', 'C_AV', 'C_FIE',
       'C_SIE', 'C_EA', 'C_EN', 'C_Np', 'C_Nv', 'C_D', 'CurrentDensity',
       'overpotential'], dtype='<U14')

In [8]:
print(tmcs['compounds'].dtype)

object


In [ ]:
train_data = []
for index, compound in enumerate(tmcs['compounds']):
    compound_feat = [compound]
    compound = get_res(compound)
    keys = list(compound.keys())
    val = list(compound.values())
    if len(keys) > 2:
        A_key, A_index = keys[0], val[0]
        B_key, B_index = keys[1], val[1]
        A_feat = tm[tm['element']==A_key].values.flatten()[1:] * float(A_index)
        B_feat = tm[tm['element']==B_key].values.flatten()[1:] * float(B_index)
        C_feat = c[c['element']==keys[2]].values.flatten()[1:] * float(val[2])
        AB_feat = np.array([a + b for a, b in zip(A_feat, B_feat)])
        compound_feat += list(AB_feat) + list(C_feat) + list([tmcs.loc[index, 'CurrentDensity'], round(tmcs.loc[index, 'overpotential'], 2)])
    else:
        AB_key = keys[0]
        AB_index = val[0]
        AB_feat = tm[tm['element']==AB_key].values.flatten()[1:] * float(AB_index)
        C_feat = c[c['element']==keys[1]].values.flatten()[1:] * float(val[1])
        compound_feat += list(AB_feat) + list(C_feat) + list([tmcs.loc[index, 'CurrentDensity'], round(tmcs.loc[index, 'overpotential'], 2)])
    train_data.append(compound_feat)
col = ['compound']
tail = ['label']
train_data_ = pd.DataFrame(train_data, columns=col + [f'feat_{i}' for i in range(1, 30)] + tail)
train_data_.to_csv('data/compound_predict_5_400.csv', index=False)


In [10]:
train_data_.columns

Index(['compound', 'feat_1', 'feat_2', 'feat_3', 'feat_4', 'feat_5', 'feat_6',
       'feat_7', 'feat_8', 'feat_9', 'feat_10', 'feat_11', 'feat_12',
       'feat_13', 'feat_14', 'feat_15', 'feat_16', 'feat_17', 'feat_18',
       'feat_19', 'feat_20', 'feat_21', 'feat_22', 'feat_23', 'feat_24',
       'feat_25', 'feat_26', 'feat_27', 'feat_28', 'feat_29', 'label'],
      dtype='object')